In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Environments

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

## Load data

In [ ]:
df= pd.read_csv('../input/churn-data-set/Churn_Train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print("=============================================================", col, len(df[col].value_counts()))
    print(df[col].value_counts())

In [ ]:
col_two = []
col_three = []
col_many = []

for col in df.columns:
    if len(df[col].value_counts()) == 2:
        col_two.append(col)
    elif len(df[col].value_counts()) == 3:
        col_three.append(col)
    else:
        col_many.append(col)

print('col_two: ', col_two)
print('col_three: ', col_three)
print('col_many: ', col_many)

### classify features

In [ ]:
discyn = ['Gender', 'Senior Citizen', 'Partner', 'Dependents','Phone Service',
         'Paperless Billing']

discval = ['Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 
           'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 
           'Contract','Payment Method']

consval = ['Tenure', 'Monthly Charges', 'Total Charges']


In [ ]:
df['Churnlbl'] = df['Churn'].apply(lambda x: 1 if x=='Yes' else 0)

### train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.copy()
X.drop(['CustomerID', 'Churn', 'Churnlbl'], axis=1, inplace=True)
y = df['Churnlbl']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

### Train data

#### discyn

In [ ]:
dft = X_train.copy()

In [ ]:
dft[discyn]

In [ ]:
en_discyn = []

dft['yn_Gender'] = dft['Gender'].apply(lambda x: 1 if x=='Male' else 0)
en_discyn.append('yn_Gender')

dft['yn_Senior Citizen'] = dft['Senior Citizen']
en_discyn.append('yn_Senior Citizen')

dft['yn_Partner'] = dft['Partner'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Partner')

dft['yn_Dependents'] = dft['Dependents'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Dependents')

dft['yn_Phone Service'] = dft['Phone Service'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Phone Service')

dft['yn_Paperless Billing'] = dft['Paperless Billing'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Paperless Billing')

In [ ]:
dft[en_discyn]

#### discval

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
dft[discval]

In [ ]:
en_discval = []

# Multiple Lines
le_Multiple_Lines = LabelEncoder().fit(dft['Multiple Lines'])
dft['en_Multiple Lines'] = le_Multiple_Lines.transform(dft['Multiple Lines'])
en_discval.append('en_Multiple Lines')

# Internet Service
le_Internet_Service = LabelEncoder().fit(dft['Internet Service'])
dft['en_Internet Service'] = le_Internet_Service.transform(dft['Internet Service'])
en_discval.append('en_Internet Service')

# Online Security
le_Online_Security = LabelEncoder().fit(dft['Online Security'])
dft['en_Online Security'] = le_Online_Security.transform(dft['Online Security'])
en_discval.append('en_Online Security')

# Online Backup
le_Online_Backup = LabelEncoder().fit(dft['Online Backup'])
dft['en_Online Backup'] = le_Online_Backup.transform(dft['Online Backup'])
en_discval.append('en_Online Backup')

# Device Protection
le_Device_Protection = LabelEncoder().fit(dft['Device Protection'])
dft['en_Device Protection'] = le_Device_Protection.transform(dft['Device Protection'])
en_discval.append('en_Device Protection')

# Tech Support
le_Tech_Support = LabelEncoder().fit(dft['Tech Support'])
dft['en_Tech Support'] = le_Tech_Support.transform(dft['Tech Support'])
en_discval.append('en_Tech Support')

# Streaming TV
le_Streaming_TV = LabelEncoder().fit(dft['Streaming TV'])
dft['en_Streaming TV'] = le_Streaming_TV.transform(dft['Streaming TV'])
en_discval.append('en_Streaming TV')

# Streaming Movies
le_Streaming_Movies = LabelEncoder().fit(dft['Streaming Movies'])
dft['en_Streaming Movies'] = le_Streaming_Movies.transform(dft['Streaming Movies'])
en_discval.append('en_Streaming Movies')

# Contract
le_Contract = LabelEncoder().fit(dft['Contract'])
dft['en_Contract'] = le_Contract.transform(dft['Contract'])
en_discval.append('en_Contract')

# Payment Method
le_Payment_Method = LabelEncoder().fit(dft['Payment Method'])
dft['en_Payment Method'] = le_Payment_Method.transform(dft['Payment Method'])
en_discval.append('en_Payment Method')

In [ ]:
dft[en_discval]

#### consval

In [ ]:
dft[consval]

In [ ]:
dft[consval].isnull().sum()

In [ ]:
dft.loc[dft['Total Charges'].isna(), 'Total Charges'] = 0

In [ ]:
for col in consval:
    print(col,'===============================================================================')
    plt.subplot(121)
    sns.distplot(dft[col], color='r')
    plt.subplot(122)
    dft[col].plot.box(figsize=(16,5))
    plt.show()

#### scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
cons_scaler = StandardScaler().fit(dft[consval].values)

In [ ]:
dft[consval] = cons_scaler.transform(dft[consval].values)

In [ ]:
dft[consval].hist(bins=50,figsize=(15,12))
plt.show()

### model

In [ ]:
Xs = dft[en_discyn + en_discval+ consval]
ys = y_train

In [ ]:
X_t, X_v, y_t, y_v= train_test_split(Xs, ys, test_size=0.2, random_state=42)

#### DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(max_depth=4)
dtree.fit(X_t, y_t)
dtree_pred = dtree.predict(X_v)

RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rftree = RandomForestClassifier(max_depth=4)
rftree.fit(X_t, y_t)
rftree_pred = rftree.predict(X_v)

### ExtraTrees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

extree = ExtraTreesClassifier(max_depth=6)
extree.fit(X_t, y_t)
extree_pred = extree.predict(X_v)

### GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbtree = GradientBoostingClassifier(max_depth=4)
gbtree.fit(X_t, y_t)
gbtree_pred = gbtree.predict(X_v)

### Metrics

In [ ]:
models = ['dtree', 'rftree', 'extree','gbtree']

preds  = ['dtree_pred', 'rftree_pred', 'extree_pred','gbtree_pred']

metric_idx = ['train_acc','test_acc']

metric = {}

for model, pred in zip(models, preds):
    metric[model] = [eval(model).score(X_t,y_t),
                     eval(model).score(X_v,y_v)
                     ]

dfmet = pd.DataFrame(metric, index=metric_idx)

dfmet.T

### Test

In [ ]:
dftt = X_test.copy()

In [ ]:
en_discyn = []

dftt['yn_Gender'] = dftt['Gender'].apply(lambda x: 1 if x=='Male' else 0)
en_discyn.append('yn_Gender')

dftt['yn_Senior Citizen'] = dftt['Senior Citizen']
en_discyn.append('yn_Senior Citizen')

dftt['yn_Partner'] = dftt['Partner'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Partner')

dftt['yn_Dependents'] = dftt['Dependents'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Dependents')

dftt['yn_Phone Service'] = dftt['Phone Service'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Phone Service')

dftt['yn_Paperless Billing'] = dftt['Paperless Billing'].apply(lambda x: 1 if x=='Yes' else 0)
en_discyn.append('yn_Paperless Billing')

In [ ]:
en_discval = []

# Multiple Lines
le_Multiple_Lines = LabelEncoder().fit(dftt['Multiple Lines'])
dftt['en_Multiple Lines'] = le_Multiple_Lines.transform(dftt['Multiple Lines'])
en_discval.append('en_Multiple Lines')

# Internet Service
le_Internet_Service = LabelEncoder().fit(dftt['Internet Service'])
dftt['en_Internet Service'] = le_Internet_Service.transform(dftt['Internet Service'])
en_discval.append('en_Internet Service')

# Online Security
le_Online_Security = LabelEncoder().fit(dftt['Online Security'])
dftt['en_Online Security'] = le_Online_Security.transform(dftt['Online Security'])
en_discval.append('en_Online Security')

# Online Backup
le_Online_Backup = LabelEncoder().fit(dftt['Online Backup'])
dftt['en_Online Backup'] = le_Online_Backup.transform(dftt['Online Backup'])
en_discval.append('en_Online Backup')

# Device Protection
le_Device_Protection = LabelEncoder().fit(dftt['Device Protection'])
dftt['en_Device Protection'] = le_Device_Protection.transform(dftt['Device Protection'])
en_discval.append('en_Device Protection')

# Tech Support
le_Tech_Support = LabelEncoder().fit(dftt['Tech Support'])
dftt['en_Tech Support'] = le_Tech_Support.transform(dftt['Tech Support'])
en_discval.append('en_Tech Support')

# Streaming TV
le_Streaming_TV = LabelEncoder().fit(dftt['Streaming TV'])
dftt['en_Streaming TV'] = le_Streaming_TV.transform(dftt['Streaming TV'])
en_discval.append('en_Streaming TV')

# Streaming Movies
le_Streaming_Movies = LabelEncoder().fit(dftt['Streaming Movies'])
dftt['en_Streaming Movies'] = le_Streaming_Movies.transform(dftt['Streaming Movies'])
en_discval.append('en_Streaming Movies')

# Contract
le_Contract = LabelEncoder().fit(dftt['Contract'])
dftt['en_Contract'] = le_Contract.transform(dftt['Contract'])
en_discval.append('en_Contract')

# Payment Method
le_Payment_Method = LabelEncoder().fit(dftt['Payment Method'])
dftt['en_Payment Method'] = le_Payment_Method.transform(dftt['Payment Method'])
en_discval.append('en_Payment Method')

In [ ]:
dftt[consval] = cons_scaler.transform(dftt[consval].values)

In [ ]:
X_t = dftt[en_discyn + en_discval+ consval]
y_t = y_test

In [ ]:
test = pd.concat([X_t,y_t], axis=1)

In [ ]:
test.isnull().sum()

In [ ]:
test.loc[test['Total Charges'].isna(),'Total Charges'] = 0

In [ ]:
X_t = test.copy()
X_t.drop('Churnlbl', axis=1, inplace=True)
y_t = test['Churnlbl']

In [ ]:
models = ['dtree', 'rftree', 'extree', 'gbtree']

for model in models:
    print('{}, {:0.6f}'.format(model, eval(model).score(X_t,y_t)))

================================================ end ===================================